In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

# Собственный трансформер категориальных переменных 
## Target Encoder

Категориальные переменные могут быть очень полезны для моделирования. Крайне важно подать их в модель наиболее эффективным способом.

Мы с вами изучили один из способов - OneHotEncoding. Такое представление может быть хорошо воспринято линейными моделями, но его [не рекомендуется](https://roamanalytics.com/2016/10/28/are-categorical-variables-getting-lost-in-your-random-forests/) использовать для деревянных методов. А с метрическими - все еще сложнее.

Сейчас мы с вами попробуем понять новый подход кодирования категориальных переменных с использованием целевой переменной - Target Encoding. Способов его реализовать [довольно много](https://towardsdatascience.com/benchmarking-categorical-encoders-9c322bd77ee8), мы рассмотрим наиболее простой из них

Рассмотрим датасет `data_train` из одной категориальной переменной `category` и целевой переменной `y`.

In [4]:
data_train = pd.DataFrame(data={
    'category': [1] * 5 + [2] * 3 + [3] * 4,
    'y': [0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1]
})

data_train

,category,y
0,1,0
1,1,0
2,1,1
3,1,1
4,1,0
5,2,1
6,2,1
7,2,0
8,3,0
9,3,0


Target encoding, как следует из названия, использует целевую переменную. Идея его довольно проста:
* Давайте для каждой категории посчитаем усредненное значение целевой переменной
* Заменим индекс категории на это значение
* Исходный столбец удаляется

Если написать это в виде формул то получим:

$$ v_c = \frac{\sum_{i \in c}y_i}{N} $$
* $v_c$ - это значение, которое получится для категории $c$
* $N$ - это общее количество объектов

Если писать это на pandas, получится следующее:

In [5]:
category_encoding = data_train.groupby('category').y.mean()
category_encoding

category
1    0.400000
2    0.666667
3    0.250000
Name: y, dtype: float64

In [6]:
data_train.loc[:, 'category_encoding'] = data_train.category.replace(category_encoding)

In [7]:
data_train.loc[:, ['category', 'category_encoding', 'y']]

,category,category_encoding,y
0,1,0.400000,0
1,1,0.400000,0
2,1,0.400000,1
3,1,0.400000,1
4,1,0.400000,0
5,2,0.666667,1
6,2,0.666667,1
7,2,0.666667,0
8,3,0.250000,0
9,3,0.250000,0


Теперь, на тестовом датасете мы получили бы такой результат:

In [8]:
data_test= pd.DataFrame(data={
    'category': [1] * 2 + [2] * 2 + [3] * 2,
})
data_test

,category
0,1
1,1
2,2
3,2
4,3
5,3


In [9]:
data_test.loc[:, 'category_encoding'] = data_test.category.replace(category_encoding, )
data_test

,category,category_encoding
0,1,0.400000
1,1,0.400000
2,2,0.666667
3,2,0.666667
4,3,0.250000
5,3,0.250000


У данного кодировщика есть одна большая проблема - он может переобучиться!

Что если какой-то категории все значения целевой переменной оказались одинаковыми? В этом случае модель будет просто копировать значения в ответ. Поэтому надо ввести **регуляризацию**.

Будем немного "ослаблять" наше кодирование - подмешаем туда общее среднее значение по целевой переменной с весом $w$. Тогда, в виде формулы получится

$$ v_c = \frac{(1-w)\sum_{i \in c}y_i + w\cdot m}{N} $$
* $m = \frac{\sum_i y_i}{N}$ - глабальное среднее по $y$
* $w\in[0,1]$ - сила регуляризации. Чем больше $w$ - тем больше будет смещение к глобальному среднему

На питоне получится как-то так:

In [10]:
w = 0.1 # Попробуйте для разных значений и сравните результаты
category_encoding = (1-w)*data_train.groupby('category').y.mean() + w*data_train.y.mean()

In [11]:
category_encoding

category
1    0.401667
2    0.641667
3    0.266667
Name: y, dtype: float64

In [12]:
data_train.loc[:, 'category_encoding'] = data_train.category.replace(category_encoding)

In [13]:
data_train.loc[:, ['category', 'category_encoding', 'y']]

,category,category_encoding,y
0,1,0.401667,0
1,1,0.401667,0
2,1,0.401667,1
3,1,0.401667,1
4,1,0.401667,0
5,2,0.641667,1
6,2,0.641667,1
7,2,0.641667,0
8,3,0.266667,0
9,3,0.266667,0


## Задача

Напишите трансформер, который делает target encoding. В качестве гиперараметра он должен принимать коэффициент $w$

Вам нужно дописать код в методы `fit` и `transform`:
* `fit` по имеющимся `X` и `y` должен "выучивать" мэппинг `self.category_mapping` категории в значение
* `transform` по имеющимся `X` должен переводить каждое значение в мэппинг

В базовой версии предполагается, что `X` состоит из одной колонки с категориальной переменной.<br>
**Бонус:** сделайте трансформер, которые будет справляться с несколькими категориальными колонками

In [62]:
from sklearn.base import BaseEstimator, TransformerMixin

class TargetEncoder(BaseEstimator, TransformerMixin):
    '''Performes target encoding of a categorical column'''
    def __init__(self, w=0.1):
        self.w = w
        self.global_mean = None
        self.category_mapping = dict()
        
    def fit(self, X, y):
        uniq = np.unique(X)
        self.global_mean = y.mean()
        
        cat_dict = {}
        for cat in uniq:
            tmp_target = [tg for x, tg in zip(X,y) if x == cat]
            cat_dict[cat] = (1-self.w)*sum(tmp_target)/len(tmp_target) + self.w*sum(y)/len(y)
        
        self.category_mapping = cat_dict
        return self

    def transform(self, X):
        return np.array([round(self.category_mapping[x[0]],8) for x in X]).reshape(-1,1)

In [50]:
X = data_train.category.values.reshape(-1,1)
y = data_train.y.values

In [66]:
# Tests

encoder = TargetEncoder(w=0.1)
X_transformed = encoder.fit_transform(X,y)
X_true = np.array([0.40166667, 0.40166667, 0.40166667, 0.40166667, 0.40166667,
                   0.64166667, 0.64166667, 0.64166667, 0.26666667, 0.26666667,
                   0.26666667, 0.26666667]).reshape(-1,1)
assert np.allclose(X_transformed, X_true)

encoder = TargetEncoder(w=0.9)
X_transformed = encoder.fit_transform(X,y)
X_true = np.array([0.415     , 0.415     , 0.415     , 0.415     , 0.415     ,
                   0.44166667, 0.44166667, 0.44166667, 0.4       , 0.4       ,
                   0.4       , 0.4       ]).reshape(-1,1)
assert np.allclose(X_transformed, X_true)

encoder = TargetEncoder(w=1)
X_transformed = encoder.fit_transform(X,y)
assert np.all(X_transformed == round(y.mean(),8))